<a href="https://colab.research.google.com/github/textspur/prompting_open_llm/blob/main/Run_phi4_in_Colab_using_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Phi-4 and other LLMs in R using Google Colab for free through the rollama R-Package

Local Large Language Models (LLMs) are becoming increasingly important for researchers and practitioners. This tutorial will show you how to run Microsoft’s Phi-4 model using R, Ollama, and the rollama package — all for free using Google Colab!

Why Colab?
- Provides easy GPU access
- No local installation required

**Setting Up Ollama in Colab**

Since we’re in Colab, we need to install Ollama in our temporary environment. This can be achieved with a few commands:

In [2]:
# Run the Ollama Linux install script
output <- system("curl -fsSL https://ollama.com/install.sh | sh", intern = TRUE)
cat(output, sep = "\n")

Start Ollama on the machine

In [3]:
# Start the Ollama service
system("ollama serve", wait = FALSE)

Check if Ollama is running:

In [4]:
# Verify Ollama is running
output <- system("curl http://localhost:11434", intern = TRUE)
cat(output, sep = "\n")

Warning message in system("curl http://localhost:11434", intern = TRUE):
“running command 'curl http://localhost:11434' had status 7”




If the previous code chunk resulted in "Ollama is running" we are good to go.

**Installing and Setting Up rollama**

The rollama package provides an interface to Ollama:

In [5]:
install.packages("rollama")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
library(rollama)

Check if ollama is running:

In [7]:
ping_ollama()

▶ Ollama (v0.5.4) is running at <http://localhost:11434>!


In order to use a generative language model, we need to download one. Ollama provides a list of supported models here: https://ollama.com/search   
Pulling a model might take a few minutes.

In [ ]:
pull_model("phi4")

**Basic Usage**

Let’s start with a simple query to test our setup. The first query after loading the model will be slower than subsequent queries.

In [ ]:
answer <- query(
 "Why is the sky blue? Answer with one sentence.",
 output = "text",
 model = "phi4",
 screen = FALSE
)
print(answer)

**Advanced Applications**

*Text Classification*

Phi-4 can be used for various NLP tasks. Here’s an example of text classification:

In [ ]:
prompt <- "Answer with just the correct category. Classify this text as either 'populist' or 'not populist':
'It's time to take back our country back from corrupt elites and put the power back where it belongs: in the hands of the people.'"

answer <- query(prompt, output = "text",  model= "phi4", screen = FALSE)
print(answer)

**Working with Data frames and a few-shot prompt**

One of the powerful features of rollama is the ability to process entire data frames. Here’s how to annotate multiple texts:

In [ ]:
unseen_texts <- c(
  "Many kids find school boring",
  "Everybody should have access to good education and healthcare"
)

df <- data.frame(text = unseen_texts)
df


This creates a data frame with a ‘text’ column. Next, we generate annotation queries using rollama’s make_query() function with two examples and a system prompt.

In [ ]:
# Create examples for few-shot learning
annotated_examples <- tibble::tribble(
 ~text, ~answer,
 "the pizza tastes terrible", "'not political'",
 "the state should make sure no child goes hungry", "'political'"
)

# Generate queries with examples
queries <- make_query(
 template = "{text}\n{prompt}",
 text = df$text,
 prompt = "Categories: 'political' or 'not political'",
 system = "You are an expert annotator who classifies texts.
 Answer with just the correct category.",
 example = annotated_examples
)

In [ ]:
print(queries)

Now we can execute the queries and store the results in a new column of the data frame:

In [ ]:
df$annotation <- query(
    q = queries,
    output = "text",
    model= "phi4",
    screen = FALSE
)
df

Remember that Colab environments are temporary. Any models you download will need to be re-downloaded in new sessions. Consider this when planning long-running tasks. Save your notebook to Google Drive to preserve your code and configurations for future use.